# Analyse des Variations d'Entrefer - Tesla Model 3

Ce notebook génère et simule 5 variations d'entrefer de la machine Tesla Model 3 pour faciliter l'analyse et le débogage.

## Objectif
- Générer 5 machines avec des variations d'entrefer cohérentes (-10%, -5%, 0%, +5%, +10%)
- Simuler ces machines pour comparer leurs performances
- Identifier les problèmes potentiels de manière ciblée


In [ ]:
# 1. Import des bibliothèques et chargement de la machine de référence

%matplotlib inline

from copy import deepcopy
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
from pyleecan.Classes.Material import Material
import matplotlib.pyplot as plt
import numpy as np

import os
from datetime import datetime

In [ ]:

print("=" * 80)
print("TESLA MODEL 3 - VARIATIONS D'ENTREFER UNIQUEMENT (5 MACHINES)")
print("=" * 80)

# Chargement de la machine de référence
ref_path = join(DATA_DIR, "Machine", "Tesla_model_3.json")
M_ref = load(ref_path)

# Valeurs de référence (vérification)
airgap_ref = M_ref.stator.Rint - M_ref.rotor.Rext
print(f"\n MACHINE DE RÉFÉRENCE:")
print(f"   Entrefer: {airgap_ref*1000:.2f} mm")
print(f"   Rayon stator intérieur: {M_ref.stator.Rint*1000:.2f} mm")
print(f"   Rayon rotor extérieur: {M_ref.rotor.Rext*1000:.2f} mm")
print(f"   Tours par bobine: {M_ref.stator.winding.Ntcoil}")
print(f"   Nombre d'encoches: {M_ref.stator.slot.Zs}")

# === FONCTION DE RESTAURATION DES MATÉRIAUX (SIMPLIFIÉE) ===
def ensure_materials_identical(M, M_ref):
    """Restaure les matériaux pour qu'ils soient identiques à la référence"""
    
    # 1. Stator - Copie exacte
    if hasattr(M_ref.stator, "mat_type") and M_ref.stator.mat_type is not None:
        M.stator.mat_type = deepcopy(M_ref.stator.mat_type)
    
    # 2. Rotor - Copie exacte
    if hasattr(M_ref.rotor, "mat_type") and M_ref.rotor.mat_type is not None:
        M.rotor.mat_type = deepcopy(M_ref.rotor.mat_type)
    
    # 3. Aimants - Copie exacte des matériaux
    if hasattr(M_ref.rotor, "hole") and M_ref.rotor.hole:
        for i, (hole_ref, hole_new) in enumerate(zip(M_ref.rotor.hole, M.rotor.hole)):
            # Traiter magnet_dict
            if hasattr(hole_ref, 'magnet_dict') and hole_ref.magnet_dict:
                for mag_key in hole_ref.magnet_dict.keys():
                    if mag_key in hole_new.magnet_dict:
                        magnet_ref = hole_ref.magnet_dict[mag_key]
                        magnet_new = hole_new.magnet_dict[mag_key]
                        
                        if magnet_ref is not None and magnet_new is not None:
                            # Copier le matériau exactement
                            if hasattr(magnet_ref, "mat_type") and magnet_ref.mat_type is not None:
                                magnet_new.mat_type = deepcopy(magnet_ref.mat_type)
            
            # Traiter les attributs magnet_*
            else:
                for attr in dir(hole_ref):
                    if attr.startswith("magnet_"):
                        magnet_ref = getattr(hole_ref, attr)
                        magnet_new = getattr(hole_new, attr)
                        
                        if magnet_ref is not None and magnet_new is not None:
                            if hasattr(magnet_ref, "mat_type") and magnet_ref.mat_type is not None:
                                magnet_new.mat_type = deepcopy(magnet_ref.mat_type)

# === FONCTION DE VALIDATION SIMPLIFIÉE ===
def validate_airgap_only(machine, name):
    """Validation uniquement pour les variations d'entrefer"""
    errors = []
    warnings = []
    
    # Vérification de l'entrefer uniquement
    airgap = machine.stator.Rint - machine.rotor.Rext
    airgap_variation = abs(airgap - airgap_ref) / airgap_ref
    
    if airgap <= 0:
        errors.append(f"Entrefer négatif: {airgap*1000:.2f} mm")
    elif airgap < 0.2e-3:  # Minimum 0.2 mm
        errors.append(f"Entrefer trop petit: {airgap*1000:.2f} mm")
    elif airgap_variation > 0.15:  # Maximum 15% de variation
        errors.append(f"Variation entrefer excessive: {airgap_variation*100:.1f}%")
    elif airgap < 0.3e-3:  # Alerte si très petit
        warnings.append(f"Entrefer très petit: {airgap*1000:.2f} mm")
    
    return errors, warnings

# === GÉNÉRATION DES 5 VARIATIONS D'ENTREFER ===
print("\n" + "=" * 60)
print("GÉNÉRATION DES 5 VARIATIONS D'ENTREFER")
print("=" * 60)

# Définition des variations d'entrefer (-10%, -5%, 0%, +5%, +10%)
airgap_variations = [-0.10, -0.05, 0.00, 0.05, 0.10]
variation_names = ["Moins10_Pourcent", "Moins5_Pourcent", "Reference", "Plus5_Pourcent", "Plus10_Pourcent"]

machines = []
names = []
airgap_values = []

print(f"\n Variations d'entrefer à générer:")
for i, (var, name) in enumerate(zip(airgap_variations, variation_names)):
    new_airgap = airgap_ref * (1 + var)
    print(f"  {i+1}. {name}: {var:+.0f}% -> {new_airgap*1000:.2f} mm")

print(f"\n Génération en cours...")

for i, (var, name) in enumerate(zip(airgap_variations, variation_names)):
    print(f"\n Génération {i+1}/5: {name}")
    
    # Créer une copie de la machine de référence
    M = deepcopy(M_ref)
    
    # Calculer le nouveau rayon rotor pour obtenir la variation d'entrefer souhaitée
    new_airgap = airgap_ref * (1 + var)
    new_Rext = M.stator.Rint - new_airgap
    
    # Appliquer UNIQUEMENT la modification du rayon rotor
    M.rotor.Rext = new_Rext
    
    # Restaurer les matériaux (copie exacte de la référence)
    ensure_materials_identical(M, M_ref)
    
    # Validation uniquement de l'entrefer
    errors, warnings = validate_airgap_only(M, name)
    
    if errors:
        print(f"    ERREUR: {errors[0]}")
        continue
    elif warnings:
        print(f"     ATTENTION: {warnings[0]}")
    else:
        print(f"    OK")
    
    # Stocker la machine
    machines.append(M)
    names.append(f"Tesla_Model3_Entrefer_{name}")
    airgap_values.append(new_airgap)

# === AFFICHAGE DES RÉSULTATS ===
print(f"\n" + "=" * 60)
print("RÉSULTATS DE LA GÉNÉRATION")
print("=" * 60)

print(f"\n Machines générées avec succès: {len(machines)}/5")
print(f"\n Détail des entrefers générés:")
for i, (name, airgap) in enumerate(zip(names, airgap_values)):
    variation = (airgap - airgap_ref) / airgap_ref * 100
    print(f"  {i+1}. {name}: {airgap*1000:.2f} mm ({variation:+.1f}%)")

# === VÉRIFICATION DE COHÉRENCE ===
print(f"\n" + "=" * 60)
print("VÉRIFICATION DE COHÉRENCE")
print("=" * 60)

if len(machines) > 0:
    # Vérifier que toutes les autres valeurs restent identiques
    print(f"\n Vérification des valeurs conservées:")
    
    # Vérifier le bobinage
    for i, (machine, name) in enumerate(zip(machines, names)):
        if machine.stator.winding.Ntcoil != M_ref.stator.winding.Ntcoil:
            print(f"    {name}: Tours par bobine différent ({machine.stator.winding.Ntcoil} vs {M_ref.stator.winding.Ntcoil})")
        else:
            print(f"    {name}: Tours par bobine identique ({machine.stator.winding.Ntcoil})")
    
    # Vérifier le nombre d'encoches
    for i, (machine, name) in enumerate(zip(machines, names)):
        if machine.stator.slot.Zs != M_ref.stator.slot.Zs:
            print(f"    {name}: Nombre d'encoches différent ({machine.stator.slot.Zs} vs {M_ref.stator.slot.Zs})")
        else:
            print(f"    {name}: Nombre d'encoches identique ({machine.stator.slot.Zs})")
    
    # Vérifier les dimensions du stator
    for i, (machine, name) in enumerate(zip(machines, names)):
        if (machine.stator.Rint != M_ref.stator.Rint or 
            machine.stator.Rext != M_ref.stator.Rext or 
            machine.stator.L1 != M_ref.stator.L1):
            print(f"    {name}: Dimensions stator modifiées")
        else:
            print(f"    {name}: Dimensions stator identiques")

# === SAUVEGARDE DES MACHINES ===
print(f"\n" + "=" * 60)
print("SAUVEGARDE DES MACHINES")
print("=" * 60)

# Créer le dossier de sauvegarde
save_dir = "Tesla_Model3_Entrefer_Variations_Uniques"
os.makedirs(save_dir, exist_ok=True)

# Sauvegarder la machine de référence
ref_save_path = os.path.join(save_dir, "Tesla_Model3_Reference.json")
M_ref.save(ref_save_path)
print(f" Machine de référence sauvegardée: {ref_save_path}")

# Sauvegarder les variations
for i, (machine, name) in enumerate(zip(machines, names)):
    save_path = os.path.join(save_dir, f"{name}.json")
    machine.save(save_path)
    print(f" Variation {i+1} sauvegardée: {save_path}")

print(f"\n" + "=" * 60)
print("GÉNÉRATION TERMINÉE AVEC SUCCÈS!")
print("=" * 60)

# === VISUALISATION DES MACHINES ===
print(f"\n" + "=" * 60)
print("VISUALISATION DES MACHINES GÉNÉRÉES")
print("=" * 60)

# Créer une figure avec les 5 machines
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Machine de référence
M_ref.plot(is_show_fig=False, ax=axes[0])
axes[0].set_title(f"Référence\n{airgap_ref*1000:.2f} mm", fontsize=12)

# Variations
for i, (machine, name, airgap) in enumerate(zip(machines, names, airgap_values)):
    if i < 5:  # Limiter à 5 variations
        machine.plot(is_show_fig=False, ax=axes[i+1])
        variation = (airgap - airgap_ref) / airgap_ref * 100
        axes[i+1].set_title(f"{name.split('_')[-2]}_{name.split('_')[-1]}\n{airgap*1000:.2f} mm ({variation:+.1f}%)", fontsize=12)

# Masquer le dernier subplot s'il n'est pas utilisé
if len(machines) < 5:
    axes[-1].set_visible(False)

plt.tight_layout()
plt.show()

# === RÉSUMÉ FINAL ===
print(f"\n" + "=" * 80)
print("RÉSUMÉ FINAL")
print("=" * 80)
print(f"  {len(machines)} machines générées avec succès")
print(f" Dossier de sauvegarde: {os.path.abspath(save_dir)}")
print(f"  Modification: Entrefer uniquement")
print(f" Valeurs conservées: Toutes les autres propriétés identiques")
print(f" Variations: -10%, -5%, 0%, +5%, +10%")
print("=" * 80)